In [1]:
import numpy as np
import copy
import os
import argparse
import math
import sys
import subprocess
import gzip

In [4]:
def get_gene_name():
    return(line_split[8].split("gene_name ")[1].split('''"''')[1])
def get_transcript_name():
    try:
        return(line_split[8].split("transcript_id ")[1].split('''"''')[1])
    except:
        print("Error: SparK tried to extract information to plot genes in the selected area from the following line in the gtf file but failed. Please check the format thoroughly, and read the SparK github page for how the gtf needs to be structured for genes to be plotted, or how to change this python script to work with this specific gtf.")
        print(line_split)
def draw_line(coordinates, thickness, color):
    output = '''<path d="'''
    for x, i in enumerate(coordinates):
        if x == 0:
            output += "M "
        else:
            output += " L "
        output += str(i[1]) + " " + str(y_start - i[0])
    output += '''" style="stroke:'''+ color + '''; stroke-width:'''+ str(thickness) +'''; fill:none "/>'''
    return(output)

def get_lines_tabix(filename, stretch):
    """
    helper function for make_raw_data_filled().
    If a data file is tabix-indexed, retrieve the desired region using tabix.
    """
    chromname = stretch[0]
    if len(stretch) > 3:
        # Chromosome name was modified. Use original name with
        # tabix index.
        chromname = stretch[3]
    tabix_cmd = subprocess.Popen(['tabix', filename, \
        '{}:{}-{}'.format(chromname, stretch[1], stretch[2])], \
        stdout=subprocess.PIPE)
    out, err = tabix_cmd.communicate()
    for line in out.decode().strip().split('\n'):
        if line != "":
            yield line

def get_lines_stream(filename):
    """
    helper function for make_raw_data_filled().
    If a data file is not tabix-indexed, go through it line by line.
    """
    if filename[-3:] == '.gz':
        f = gzip.open(filename, 'r')
        for line in f:
            yield line.rstrip()
    else:
        f = open(filename, 'r')
        for line in f:
            yield line.rstrip()
    f.close()

def make_raw_data_filled(stretch, files, offset):  # files[ctrl,treat]
    raw_data_filled = [[0] * (stretch[2] - stretch[1]) for r in range(len(files))]
    for a, datafile2 in enumerate(files):
        # Check to see whether the file is tabix-indexed. If so, use index for
        # faster processing
        input_lines = None
        found_chromosome = False
        if os.path.isfile('{}.tbi'.format(datafile2)):
            input_lines = get_lines_tabix(datafile2, stretch)
            # Lines will already be in the specified interval.
            found_chromosome = True
        else:
            print("WARNING: {} is not tabix-indexed.".format(datafile2), \
                file=sys.stderr)
            print("To speed up, compress with bgzip and index with tabix.", \
                file=sys.stderr)
            input_lines = get_lines_stream(datafile2)
        
        for line in input_lines:
            line_split = line.split("\t")
            try:
                if line_split[0][:3] == "chr" or line_split[0][:3] == "Chr":
                    line_split[0] = line_split[0][3:]
            except:
                pass
            if found_chromosome and line_split[0] != stretch[0]:
                break

            if line_split[0] == stretch[0]:
                try:
                    line_split[3] = float(line_split[3].split("\n")[0])
                except:
                    print("Warning! Could not import following row from: " + datafile2)
                    print(line)
                    print("Continuing to Import...")
                    print("")
                found_chromosome = True
                if int(line_split[2]) >= stretch[1] + offset:
                    if int(line_split[1]) <= stretch[2] + offset:
                        for iteration in range(int(line_split[2]) - int(line_split[1])):
                            try:
                                raw_data_filled[a][int(line_split[1]) + iteration - stretch[1] + offset] = line_split[3]
                            except:
                                pass

    # shrink to max_datapoints if bigger
    max_datapoints = 2000
    if stretch[2] - stretch[1] > max_datapoints:
        binfactor_split = math.modf(float((float(stretch[2] - stretch[1]))/max_datapoints))  # get values after and before period
        binfactor = sum(binfactor_split)
        temp_data = [[] for u in range(len(files))]  # new data list
        for workingfilenr in range(len(files)):
            for position in range(max_datapoints):
                start_postition_split = math.modf(position * binfactor)  # after and before period

                # first add fraction of start position or entire value if no fraction
                temp_value = float(raw_data_filled[workingfilenr][(int(start_postition_split[1]))] * (1 - start_postition_split[0]))
                binfactor_left = binfactor - (1 - start_postition_split[0])

                # add all values with no fractions
                iteration = 0
                while binfactor_left > 1:
                    temp_value += raw_data_filled[workingfilenr][int(start_postition_split[1]) + 1 + iteration]
                    iteration += 1
                    binfactor_left -= 1

                # add last fraction or value if no fraction
                if binfactor_left > 0:
                    if float((start_postition_split[1]) + 1 + iteration) < len(raw_data_filled[0]):
                        temp_value += raw_data_filled[workingfilenr][int(start_postition_split[1]) + 1 + iteration] * binfactor_left
                        temp_data[workingfilenr].append(temp_value/sum(binfactor_split))
        raw_data_filled = copy.deepcopy(temp_data)

    if smoothen_tracks is not None:
        raw_data_filled_smooth = [[0] * 2000 for r in range(len(files))]
        for x, dataset in enumerate(raw_data_filled):
            temp = [dataset[0]] * smoothen_tracks
            for p, i in enumerate(dataset):
                temp.append(i)
                temp.pop(0)
                raw_data_filled_smooth[x][p] = np.average(temp)
        raw_data_filled = copy.deepcopy(raw_data_filled_smooth)
    return raw_data_filled
def write_to_file(row):
    with open(output_filename, "a") as f:
        f.write(row)
        f.write("\n")
def get_max_value(datasets1, datasets2):
    plottingaverages = False
    if show_plots == "averages":
        plottingaverages = True
    max_1 = []
    for datafile1 in datasets1:
        max_1.append(max(datafile1))
    max_2 = []
    for datafile2 in datasets2:
        max_2.append(max(datafile2))
    if plottingaverages == True:
        if max_2 != []:
            if max_2 != []:
                return max([np.average(max_1), np.average(max_2)])
            else:
                return(np.average(max_1))
    elif plottingaverages == False:
        if max_2 != []:
            return max([max(max_1), max(max_2)])
        else:
            return(max(max_1))
def get_relative_hight(raw_value): # FIX make sure maxvalue can be 0 too
    if raw_value == 0:
        return(0)
    else:
        return((raw_value * hight * relative_track_hight_percentage) / max_value) # to not go up to the max
def draw_rect(x_coord, y_0, color, width, hight1, opacity):
    return '''<rect x="''' + str(x_coord) + '''" opacity="''' + str(opacity) + '''" y="''' + str(y_0 - hight1) + '''" fill="''' + color + '''" width="''' + str(width) + '''" height="''' + str(hight1) + '''"/>'''
def draw_polygon(coordinates, opacity, color, stroke_width):
    for q, w in enumerate(coordinates):
        coordinates[q][0] = y_start - coordinates[q][0]
    string = '''<polygon points="'''
    for h, c in enumerate(coordinates):
        if h == 0:
            string += str(c[1]) + "," + str(c[0])
        else:
            string += " " + str(c[1]) + "," + str(c[0])
    string += '''" opacity="''' + str(opacity) + '''" fill="''' + color + '''"''' + ''' stroke="black" stroke-width="''' + str(stroke_width) + '''"/>'''
    return string
def draw_standard_spark():
    if len(control_data) > 1 and len(treat_data) > 1:
        last_xpos = -1
        coords = []  # y/x, spark color
        last_value = ""
        for x, value in enumerate(control_data[0]):
            x_pos = x_start + (x * quantile)  # y/x
            ctrl_values = []
            treat_values = []
            for p, i in enumerate(control_data):
                ctrl_values.append(control_data[p][x])
            for p, i in enumerate(treat_data):
                treat_values.append(treat_data[p][x])

            sum_std = np.std(ctrl_values) + np.std(treat_values)
            if abs(np.average(ctrl_values) - np.average(treat_values)) > sum_std:
                if np.average(ctrl_values) > np.average(treat_values):
                    if last_value == "" or last_value == "up":
                        if (last_xpos + 1) == x:
                            coords.append([get_relative_hight(np.average(ctrl_values) - np.std(ctrl_values)), x_pos])
                            coords.insert(0, [get_relative_hight(np.average(treat_values) + np.std(treat_values)), x_pos])
                            last_xpos = x
                        else:
                            if len(coords) > 0:
                                write_to_file(draw_polygon(coords, 0.8, spark_color[1], stroke_width_spark))
                            coords = [[get_relative_hight(np.average(ctrl_values) - np.std(ctrl_values)), x_pos]]
                            coords.insert(0, [get_relative_hight(np.average(treat_values) + np.std(treat_values)), x_pos])
                            last_xpos = x
                            last_value = "up"
                    else:
                        if len(coords) > 0:
                            write_to_file(draw_polygon(coords, 0.8, spark_color[0], stroke_width_spark))
                        coords = [[get_relative_hight(np.average(ctrl_values) - np.std(ctrl_values)), x_pos]]
                        coords.insert(0, [get_relative_hight(np.average(treat_values) + np.std(treat_values)), x_pos])
                        last_xpos = x
                        last_value = "up"

                if np.average(ctrl_values) < np.average(treat_values):
                    if last_value == "" or last_value == "down":
                        if (last_xpos + 1) == x:
                            coords.append([get_relative_hight(np.average(treat_values) - np.std(treat_values)), x_pos])
                            coords.insert(0, [get_relative_hight(np.average(ctrl_values) + np.std(ctrl_values)), x_pos])
                            last_xpos = x
                        else:
                            if len(coords) > 0:
                                write_to_file(draw_polygon(coords, 0.8, spark_color[0], stroke_width_spark))
                            coords = [[get_relative_hight(np.average(treat_values) - np.std(treat_values)), x_pos]]
                            coords.insert(0, [get_relative_hight(np.average(ctrl_values) + np.std(ctrl_values)), x_pos])
                            last_xpos = x
                            last_value = "down"
                    else:
                        if len(coords) > 0:
                            write_to_file(draw_polygon(coords, 0.8, spark_color[1], stroke_width_spark))
                        coords = [[get_relative_hight(np.average(treat_values) - np.std(treat_values)), x_pos]]
                        coords.insert(0, [get_relative_hight(np.average(ctrl_values) + np.std(ctrl_values)), x_pos])
                        last_xpos = x
                        last_value = "down"
        if len(coords) > 0:
            if last_value == "up":
                write_to_file(draw_polygon(coords, spark_opacity, spark_color[0], stroke_width_spark))
            elif last_value == "down":
                write_to_file(draw_polygon(coords, spark_opacity, spark_color[1], stroke_width_spark))
    else:
        print("Error: Some Sparks not plotted as sparks require at least 2 control and treatment files per plot")
def get_region_to_draw():
    region_to_draw = 0
    if line_split[4] > region[1] and line_split[3] < region[2]: # check if there is something to draw
        region_to_draw = [float(line_split[3]), float(line_split[4])]
        if line_split[3] < region[1]:
            region_to_draw[0] = float(region[1])
        if line_split[4] > region[2]:
            region_to_draw[1] = float(region[2])
    return(region_to_draw)


# Additional Arguments #########################################
hight = 30  # hight of plots
x_start = 50
spark_opacity = 1
stroke_width = 0  # 0.1 stroke widths good
stroke_width_spark = 0
plot_all_TSS = False  ## could plot all TSS sites
relative_track_hight_percentage = 0.9

In [3]:
%tb

SystemExit: 2